In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/Data_Challenge/data.zip" -d /content/
%cd content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from load_data import load_train_data
from text_encoding import embed_texts
from training import train_clip_with_split
from eval_submission import build_submission_df

# Load data

In [ ]:
motion_ids, motion_paths, map_df, text_df = load_train_data("./")

# Encode text

In [ ]:
text_encoder = "BAAI/bge-m3"

text_df, text_emb, _ = embed_texts(text_df, model_name=text_encoder)

# Motion encoder

In [ ]:
from motion_encoder.lstm import LSTMMotionEncoder
from motion_encoder.simple_mlp import SimpleMotionEncoder

In [ ]:
x0 = np.load(motion_paths[0])
d_in = x0.shape[1]

motion_model = LSTMMotionEncoder(d_in=d_in, hidden_dim=256, out_dim=512)

# Training

In [ ]:
trained_clip, info = train_clip_with_split(
    motion_model=motion_model,
    motion_ids=motion_ids,
    motion_paths=motion_paths,
    map_df=map_df,
    text_df=text_df,
    text_emb=text_emb,
    text_model_name=text_encoder,
    epochs=50,
    batch_size=32,   # lower if VRAM tight
)

# Eval & build submission

In [ ]:
pred_df = build_submission_df(trained_clip, test_root="./test/test", top_k=10)
pred_df.head()

In [ ]:
pred_df.to_csv("./submission_bge-m3_LSTM2.csv", index=False)